# Exploration 노드 14
## 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np

from implicit.als import AlternatingLeastSquares


## 1) 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 데이터 탐색

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 가장 인기 있는 영화 30개 (인기순)
movie_count = ratings.groupby('movie_id')['user_id'].count()
top_30 = movie_count.sort_values(ascending=False).head(30)
top_30

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [10]:
# top 30 영화 이름으로 변환
movies.set_index('movie_id').loc[top_30.index]

,title,genre
movie_id,,
2858,American Beauty (1999),Comedy|Drama
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2028,Saving Private Ryan (1998),Action|Drama|War
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
593,"Silence of the Lambs, The (1991)",Drama|Thriller
1198,Raiders of the Lost Ark (1981),Action|Adventure
1270,Back to the Future (1985),Comedy|Sci-Fi


## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가

In [11]:
# 좋아하는 영화 설정
my_favorite = [1, 480, 1270, 1580, 2571] #Toy Story (1995), Jurassic Park (1993), Jurassic Park (1993), Men in Black (1997), Matrix, The (1999)

# 'hyunbin'이라는 user_id로 위 영화를 각 5점 평점 준 걸로 가정
my_ratings = pd.DataFrame({'user_id': ['hyunbin']*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['hyunbin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_ratings)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,hyunbin,1,5,NaN
1,hyunbin,480,5,NaN
2,hyunbin,1270,5,NaN
3,hyunbin,1580,5,NaN
4,hyunbin,2571,5,NaN


### indexing 작업
movie_id, user_id모두 정수로 표현되어 있긴 하다. 하지만, user_id의 unique한 개수는 6039이지만 ratings.tail()에서 볼 수 있듯이 6040인 id도 존재하는 것으로 보아, 6040까지 순서대로 존재하는 것이 아닐 수도 있겠다는 생각이 들어서 안전하게 indexing작업을 다시 해준다. 

In [12]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['hyunbin'])    # 본인 추가 이전에 6039명이었으니까 6040의 인덱스 6039가 나올 것
print(movie_to_idx[1])

6039
40


In [14]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,40,5,NaN
1,6039,107,5,NaN
2,6039,22,5,NaN
3,6039,175,5,NaN


## 4) CSR matrix 만들기

In [15]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

user_id 6040개와 영화 종류 3628, 이전에 확인한 바와 같이 CSR matrix가 바르게 만들어졌다. 

## 5) Alternating Least Squares 모델 구성 및 훈련

In [16]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 훈련된 모델이 예측한 본인의 Toy Story에 대한 선호도 파악
이전에 선호하는 영화 5개 중 하나였던 Toy Story에 대해 선호도를 예측해본다

In [20]:
hyunbin, toy_story = user_to_idx['hyunbin'], movie_to_idx[1]
hyunbin_vector, toy_story_vector = als_model.user_factors[hyunbin], als_model.item_factors[toy_story]

In [21]:
hyunbin_vector

array([-1.1383982 , -0.07819616,  0.37453967,  0.15455124, -0.51168364,
        0.06617605, -0.39445716, -0.78122103,  0.78242964,  0.77877754,
       -0.6330983 , -0.8130808 , -0.65025187, -0.05449129, -0.06789885,
       -1.16943   , -0.7016614 , -0.4064988 ,  0.07528808, -0.76124954,
        0.6175956 ,  0.888439  ,  0.27277023,  0.44754192, -0.986833  ,
        0.02152161,  0.9181541 ,  0.27770656,  1.1259314 , -0.07570462,
        0.8577721 ,  0.5044441 , -0.5530434 , -0.5083939 ,  0.37362745,
       -0.4982296 , -0.178543  ,  0.58219755,  0.00216275, -0.78056246,
       -0.37530708, -0.06247796,  0.11998229, -0.10823387,  0.23147298,
        0.6867896 , -0.02596682, -0.74093235, -0.9932378 ,  0.44862765,
       -0.53522444, -0.62469   , -0.12294329, -0.218166  , -0.00460109,
       -0.42616266, -0.71186846,  0.45380276,  0.29984584,  0.92870724,
       -0.6520201 ,  1.0518672 ,  0.01143367,  0.32822376, -0.12964441,
        0.14218715,  1.1833324 , -0.1017923 ,  0.00722257,  0.24

In [22]:
toy_story_vector

array([-0.01675921, -0.00014212,  0.0135762 ,  0.00538307, -0.0006076 ,
        0.00478166, -0.01530477, -0.02189384,  0.02232796,  0.03698162,
       -0.04230478, -0.00218899,  0.02180272, -0.01681834,  0.01292478,
       -0.01653754, -0.01077965,  0.0113485 ,  0.00717524,  0.01619947,
        0.01646666,  0.0040474 ,  0.03990626, -0.00752267,  0.00178176,
        0.0411899 ,  0.02812581,  0.03875994,  0.02855791,  0.03304703,
        0.03579781,  0.01574033, -0.00556622, -0.01999954,  0.01098441,
        0.00388929, -0.01396618,  0.00726561,  0.00811704, -0.00676703,
        0.01333155, -0.00814719,  0.01081152,  0.01189251, -0.02310191,
        0.03117475,  0.00504112, -0.02338196, -0.012821  ,  0.00345119,
       -0.02718284,  0.02003157, -0.02184175, -0.01018729, -0.00368453,
        0.00220701, -0.01504678, -0.01706516,  0.01485381,  0.01089784,
       -0.01476017,  0.04054892, -0.00612459,  0.01697828, -0.0358392 ,
       -0.00602869,  0.01691697,  0.05625984,  0.02882685,  0.02

In [23]:
np.dot(hyunbin_vector, toy_story_vector)

0.50616264

### 다시 훈련
생각보다 낮은 수치가 나와 factors를 증가시켜 모델을 다시 훈련시키고 예측한다. 

In [24]:
# factors 늘려서 다시 훈련
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [25]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [26]:
hyunbin, toy_story = user_to_idx['hyunbin'], movie_to_idx[1]
hyunbin_vector, toy_story_vector = als_model.user_factors[hyunbin], als_model.item_factors[toy_story]

In [27]:
np.dot(hyunbin_vector, toy_story_vector)

0.6663924

factors를 200으로 늘렸더니 더 높은 수치로 예측이 되었다. 

## 7) 좋아하는 영화와 비슷한 영화 추천받기

In [28]:
movie_to_title = movies.set_index('movie_id')['title'].to_dict()
movie_to_title

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (1995)',
 33: 'Wings of Courage (1995)',
 34: 'Babe (1

In [29]:
favorite_movie = 480  # Jurassic Park
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(107, 1.0000001),
 (175, 0.563765),
 (92, 0.48172498),
 (82, 0.41306582),
 (2748, 0.3728444),
 (62, 0.36955008),
 (3100, 0.3575609),
 (1754, 0.35660875),
 (3297, 0.35640624),
 (3482, 0.35610554),
 (2714, 0.3545972),
 (3289, 0.3540465),
 (3510, 0.3523369),
 (3424, 0.3521799),
 (3439, 0.35127592)]

In [30]:
#movie_to_idx 를 뒤집어, index로부터 movie_id를 얻는 dict를 생성 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[movie_to_title[idx_to_movie[i[0]]] for i in similar_movie]

['Jurassic Park (1993)',
 'Men in Black (1997)',
 'Terminator 2: Judgment Day (1991)',
 'Lost World: Jurassic Park, The (1997)',
 'Bewegte Mann, Der (1994)',
 'Total Recall (1990)',
 'Little City (1998)',
 'Little Odessa (1994)',
 'Germinal (1993)',
 "Pot O' Gold (1941)",
 'Frankie Starlight (1995)',
 'Nekromantik (1987)',
 'Bittersweet Motel (2000)',
 'Bushwhacked (1995)',
 'Vampyros Lesbos (Las Vampiras) (1970)']

## 가장 좋아할 만한 영화 추천받기

In [31]:
user = user_to_idx['hyunbin']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 0.6144207),
 (50, 0.3487953),
 (62, 0.33419937),
 (141, 0.2736295),
 (145, 0.23496845),
 (372, 0.21852124),
 (117, 0.21574768),
 (44, 0.21536069),
 (200, 0.21374393),
 (375, 0.20837067),
 (479, 0.19689825),
 (82, 0.19323581),
 (51, 0.19218412),
 (317, 0.18911493),
 (110, 0.18570706),
 (154, 0.18176852),
 (150, 0.1777291),
 (60, 0.16972637),
 (58, 0.16835584),
 (38, 0.16820127)]

In [32]:
[movie_to_title[idx_to_movie[i[0]]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Toy Story 2 (1999)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Fifth Element, The (1997)',
 'X-Men (2000)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Terminator, The (1984)',
 'Face/Off (1997)',
 'Contact (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Fargo (1996)',
 'Twelve Monkeys (1995)',
 'Groundhog Day (1993)',
 'As Good As It Gets (1997)',
 'Independence Day (ID4) (1996)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Mission: Impossible (1996)',
 'Sixth Sense, The (1999)']

### 영화가 추천에 기영한 정도 확인

In [33]:
# X-Men 추천에 기영한 영화의 기여 정도 확인
[movie_to_title[idx_to_movie[i[0]]] for i in movie_recommended].index('X-Men (2000)')

5

In [34]:
movie_recommended[11]

(82, 0.19323581)

In [35]:
xmen = 372
explain = als_model.explain(user, csr_data, itemid=xmen)

In [36]:
[(movie_to_title[idx_to_movie[i[0]]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.09648291407921478),
 ('Men in Black (1997)', 0.060668744643062864),
 ('Jurassic Park (1993)', 0.039601244838902885),
 ('Back to the Future (1985)', 0.038127360912773245),
 ('Toy Story (1995)', -0.023835963165331424)]

가장 크게 영향 미친 영화들이 Matrix와 Men in Black 정도로 나온다. 모두 액션 영화이면서 sci-fi 장르라는 점에서 공통점이 있어서가 아닐까 싶다. 

# 회고

* 처음으로 추천시스템 관련하여 실습을 해보게 되었다. 사실 여태까지 CV나 NLP 위주로 공부를 했어서 항상 시계열이나 추천 관련해서 어떻게 모델이 어떤 데이터로 어떻게 학습하는가에 대해 궁금했는데, 최근에 다 직접 다루어볼 수 있게 되어서 재미 있었다. 
* 사실 Alternating Least Squares의 모델이 작동하는 원리나 각 파라미터가 어떤 역할을 하는 지에 대해 잘은 모르겠다. 노드에서 filter나 iteration을 증가시키면 성능 향상이 있을 것이라고 해서 그대로 했더니 성능이 오르긴 했지만 왜 그런지에 대한 이해가 아직 부족하다. 따로 공부를 추가적으로 해야될 부분이다. 
* 이번 노드를 하면서 느낀 것은, 데이터의 형식이나 모델의 파라미터에 있어서 크게 건들 수 있는 부분이 별로 없다고 느꼈다. 아직 MNIST 데이터셋 다루듯이 기초적인 개념을 다룬 것이기 때문에 그런 것이겠지만.. 좀 더 심화적인 개념으로 가면 어떤 식으로 추천 시스템을 구현할 지 너무 궁금하다. 